In [ ]:
# Attempt to solve QOSF monthly challenge march 2022 Task 1: https://github.com/qosf/monthly-challenges/blob/main/challenge-2022.03-mar/challenge-2022.03-mar.ipynb
import math
from collections import Counter

from qiskit import IBMQ, BasicAer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.visualization import plot_histogram
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector

from typing import List, Tuple
import numpy as np
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, execute
from qiskit_optimization import QuadraticProgram
from qiskit.circuit.quantumregister import QuantumRegister
from qiskit.circuit.classicalregister import ClassicalRegister
from qiskit.circuit.library import GroverOperator, ZGate
from qiskit.algorithms import AmplificationProblem

from qiskit_optimization.translators import from_docplex_mp
from qiskit.circuit.library.arithmetic.adders.draper_qft_adder import DraperQFTAdder
from qiskit.algorithms import Grover
from qiskit import Aer

In [ ]:
def get_registers(n, B):
    qregs=[] # quantum registers
    cregs=[] # classical registers
    for i in range(1):
        qregs.append(QuantumRegister(B, "e"+str(i)))
        cregs.append(ClassicalRegister(B, "ce"+str(i)))
    qregs.append(QuantumRegister(B, "res")) # result qubit
    cregs.append(ClassicalRegister(B, "cr")) # result qubit
    for i in range(n):
        qregs.append(QuantumRegister(1, "addr"+str(i)))
        cregs.append(ClassicalRegister(1, "caddr"+str(i)))
        
    return qregs, cregs

def apply_h_on_address(c, n, qregs, B):
    for reg in qregs:
        if 'addr' in reg.name:
            c.h(reg)
        # c.h(qregs[B+1+i])
        

def set_values(c, arr, B, n, qregs):
    for i in range(len(arr)):
        val = arr[i]
        bin_val = bin(val)[2:]
        # do some padding
        bin_val = "0"*(B-len(bin_val))+bin_val
        ctrl = B*(n+1)+i
        for j, bin_digit in enumerate(bin_val):
            if bin_digit == "1":
                c.cx(ctrl,qregs[i][B-j-1])
                

def add_all(c, n, B, R, sign):
    for i in range(n):
        c = add(c, B, range(i*B, (i+1)*B), R, sign)
    return c

def add(c, B, q1, q2, sign):
    draper_circ = DraperQFTAdder(B)
    if sign == 1:
        c = c.compose(draper_circ, list(q1) + list(q2))
    elif sign == -1:
        c = c.compose(draper_circ.inverse(), list(q1) + list(q2))
    return c

def set_value_and_add_to_reg_and_unset_values(i, c, arr, B, n, R, qregs, sign):
    val = arr[i] 
    bin_val = bin(val)[2:]
    # do some padding
    bin_val = "0"*(B-len(bin_val))+bin_val
    print(qregs)
    # set values
    for idx, reg in enumerate(qregs):
        if reg.name == "addr"+str(i):
            ctrl = idx
            for j, bin_digit in enumerate(bin_val):
                if bin_digit == "1":
                    c.cx(qregs[idx][0], qregs[0][B-j-1])
    # add
    c = add(c, B, qregs[0], range(R[0]*B, R[0]*B+1*B), sign)
    # unset
    for idx, reg in enumerate(qregs):
        if reg.name == "addr"+str(i):
            ctrl = idx
            for j, bin_digit in enumerate(bin_val):
                if bin_digit == "1":
                    c.cx(qregs[idx][0], qregs[0][B-j-1])

    return c
    
            
    
        
def grovers_shit(c, n, B, R, target_sum, grover_iterations, qregs):
    # GET THE ORACLE
    oracle = QuantumCircuit(*qregs)
    # part 2 set values, add to reg, unset_values
    for i in range(n):
        oracle = set_value_and_add_to_reg_and_unset_values(i, oracle, arr, B, n, R, qregs, 1)
        oracle.barrier()
    # # part 3 oracle check
    t_bin = bin(target_sum)[2:]
    t_bin = "0"*(B-len(t_bin))+t_bin
    print(target_sum)
    for i, bin_digit in enumerate(t_bin):
        print(i, bin_digit, "i")
        if str(bin_digit) == "0":
            print(1*B+B-i-1)
            print("here")
            oracle.x(1*B+B-i-1)
    oracle.append(ZGate().control(B-1), range(1*B,1*B+B))
    for i, bin_digit in enumerate(t_bin):
        if bin_digit == "0":
            oracle.x(1*B+B-i-1)
    # # part 5 set value, subtract from register, unsetvalue
    for i in range(n):
        oracle = set_value_and_add_to_reg_and_unset_values(i, oracle, arr, B, n, R, qregs, -1)
        oracle.barrier()

    # Now build the Diffuser
    diffuser = QuantumCircuit(n) # just the address qubits
    for j in range(n):
        diffuser.h(j)
        diffuser.x(j)
    LAST = n-1
    diffuser.h(LAST)
    # ccnot (multi control Tofolli gate)
    diffuser.mct(list(range(0, n-1)), LAST)
    diffuser.h(LAST)
    for j in range(n):
        diffuser.x(j)
        diffuser.h(j)
    
    for i in range(grover_iterations):
    #     c = c.compose(diffuser, range(n*B+B, n*B+B+n))
        c = c.compose(oracle)
        c.barrier()
        c = c.compose(diffuser, range(2*B, 2*B+n))
        c.barrier()
    return c

        
        
def get_circ(arr, B, target_sum, grover_iterations):
    n = len(arr)
    addr=[0]*n
    qregs, cregs = get_registers(n, B)
    
    R = []
    for idx, reg in enumerate(qregs):
        if "res" in reg.name:
            R.append(idx)
        
    c = QuantumCircuit(*qregs, *cregs)
    apply_h_on_address(c, n, qregs, B)
    c.barrier()
    # GROVERS SHIT
    c = grovers_shit(c, n, B, R, target_sum, grover_iterations, qregs)
    # c.barrier()
    c.measure(range(0, 2*B+n), range(0, 2*B+n))
    return c


arr = [1, 3, 6, 4, 2]
B = len(bin(max(arr))) - 2 # bit_length
k=3
# N=2**(2*4+3)
N=2**B
grover_iterations = int(math.sqrt(N/k)*math.pi/4)
c = get_circ(arr, B, 6, grover_iterations)
c.draw('mpl')


In [ ]:
backend = BasicAer.get_backend('qasm_simulator')
shots = 1000
results = execute(c, backend=backend, shots=shots).result()
answer = results.get_counts()

print(Counter(answer).most_common())
# print(answer)
plot_histogram(answer)
